# **Task 4: Regularized Regression (Ridge & Lasso)**

In [217]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [243]:
df = pd.read_csv(r'C:\Users\HP\Desktop\train.csv')
df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [244]:
df['date'] = pd.to_datetime(df['date'])
print(df['date'].dtypes)

datetime64[ns]


In [245]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day_of_year
print(df.head(2))

   id       date  store_nbr      family  sales  onpromotion  year  month  day
0   0 2013-01-01          1  AUTOMOTIVE    0.0            0  2013      1    1
1   1 2013-01-01          1   BABY CARE    0.0            0  2013      1    1


In [246]:
df['holiday'] = df['day'].apply(lambda x:1 if x ==0 else 0)
print(df['holiday'].value_counts())

holiday
0    3000888
Name: count, dtype: int64


In [247]:
print(df['family'].unique())

print(df['family'].nunique())

['AUTOMOTIVE' 'BABY CARE' 'BEAUTY' 'BEVERAGES' 'BOOKS' 'BREAD/BAKERY'
 'CELEBRATION' 'CLEANING' 'DAIRY' 'DELI' 'EGGS' 'FROZEN FOODS' 'GROCERY I'
 'GROCERY II' 'HARDWARE' 'HOME AND KITCHEN I' 'HOME AND KITCHEN II'
 'HOME APPLIANCES' 'HOME CARE' 'LADIESWEAR' 'LAWN AND GARDEN' 'LINGERIE'
 'LIQUOR,WINE,BEER' 'MAGAZINES' 'MEATS' 'PERSONAL CARE' 'PET SUPPLIES'
 'PLAYERS AND ELECTRONICS' 'POULTRY' 'PREPARED FOODS' 'PRODUCE'
 'SCHOOL AND OFFICE SUPPLIES' 'SEAFOOD']
33


In [248]:
df = df.drop(columns=['id','date', 'family'])

In [249]:
df['day_lag_1'] = df['sales'].shift(1)
df['day_lag_30'] = df['sales'].shift(30)
df = df.dropna()

In [250]:
df['rolling_mean_7']=df['sales'].rolling(window=7).mean()
df['rolling_std_7']=df['sales'].rolling(window=7).std()
df = df.dropna()

**no matter how many features i try the data was too clean. i wan getting same metrics from both models. so i am now going to add some noise by myself to distinguish between the model clearly.**

In [251]:
df['lag_1_dup'] = df['day_lag_1'] * 1.0  # perfect duplicate
df['lag_30_dup'] = df['day_lag_30'] * 1.0
df['lag_1_noise'] = df['day_lag_1'] + np.random.normal(0, df['day_lag_1'].std()*0.1, size=len(df))


In [252]:
y = df['sales']
x = df.drop(columns=['sales'])


time series data can't be shuffle or split randomly

In [253]:
from sklearn.model_selection import train_test_split
train_size = int(0.8*len(x))
x_train = x.iloc[:train_size]
x_test = x.iloc[train_size:]

y_train = y.iloc[:train_size]
y_test = y.iloc[train_size:]



In [254]:
print(x_train.head())

    store_nbr  onpromotion  year  month  day  holiday  day_lag_1  day_lag_30  \
36         10            0  2013      1    1        0        0.0         0.0   
37         10            0  2013      1    1        0        0.0         0.0   
38         10            0  2013      1    1        0        0.0         0.0   
39         10            0  2013      1    1        0        0.0         0.0   
40         10            0  2013      1    1        0        0.0         0.0   

    rolling_mean_7  rolling_std_7  lag_1_dup  lag_30_dup  lag_1_noise  
36             0.0            0.0        0.0         0.0  -265.255277  
37             0.0            0.0        0.0         0.0    58.438925  
38             0.0            0.0        0.0         0.0  -136.265201  
39             0.0            0.0        0.0         0.0   196.061283  
40             0.0            0.0        0.0         0.0  -130.238061  


In [255]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

**Linear Regression Model**

In [256]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x_train_scaled, y_train)

,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False


**making the prediction from the model in test data**

In [257]:
y_pred = model.predict(x_test_scaled)

**Metrics to evaluate the model**

In [258]:
from sklearn import metrics 
mse = metrics.mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("Root Mean Squared Error :", rmse)

mae = metrics.mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error :", mae)

r2_score = metrics.r2_score(y_test, y_pred)
print("R2 score :", r2_score)

Root Mean Squared Error : 957.6969378054148
Mean Absolute Error : 491.2069189739607
R2 score : 0.5051560880575434


so we get some key metrics of the baseline model which we will use to analyze with other models to findout which model performs the best.

In [259]:
coeffecients = pd.DataFrame({
    'Features' : x.columns,
    'Coeffecients' : model.coef_

})
print(coeffecients)

          Features  Coeffecients
0        store_nbr -2.884590e+00
1      onpromotion  2.192173e+02
2             year -2.682708e+01
3            month  1.624855e+01
4              day -2.127455e+01
5          holiday -2.273737e-13
6        day_lag_1 -1.470812e+02
7       day_lag_30 -6.682189e+01
8   rolling_mean_7  1.315736e+03
9    rolling_std_7 -7.550091e+02
10       lag_1_dup -1.470812e+02
11      lag_30_dup -6.682189e+01
12     lag_1_noise  1.010818e-01


# Now importing and fitting the Ridge Regression

In [268]:
from sklearn.linear_model import Ridge
ridge_model = Ridge(alpha=1)

ridge_model.fit(x_train_scaled, y_train)

,"alpha alpha: {float, ndarray of shape (n_targets,)}, default=1.0Constant that multiplies the L2 term, controlling regularizationstrength. `alpha` must be a non-negative float i.e. in `[0, inf)`.When `alpha = 0`, the objective is equivalent to ordinary leastsquares, solved by the :class:`LinearRegression` object. For numericalreasons, using `alpha = 0` with the `Ridge` object is not advised.Instead, you should use the :class:`LinearRegression` object.If an array is passed, penalties are assumed to be specific to thetargets. Hence they must correspond in number.",1
,"fit_intercept fit_intercept: bool, default=TrueWhether to fit the intercept for this model. If setto false, no intercept will be used in calculations(i.e. ``X`` and ``y`` are expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"max_iter max_iter: int, default=NoneMaximum number of iterations for conjugate gradient solver.For 'sparse_cg' and 'lsqr' solvers, the default value is determinedby scipy.sparse.linalg. For 'sag' solver, the default value is 1000.For 'lbfgs' solver, the default value is 15000.",None
,"tol tol: float, default=1e-4The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for each solver:- 'svd': `tol` has no impact.- 'cholesky': `tol` has no impact.- 'sparse_cg': norm of residuals smaller than `tol`.- 'lsqr': `tol` is set as atol and btol of scipy.sparse.linalg.lsqr, which control the norm of the residual vector in terms of the norms of matrix and coefficients.- 'sag' and 'saga': relative change of coef smaller than `tol`.- 'lbfgs': maximum of the absolute (projected) gradient=max|residuals| smaller than `tol`... versionchanged:: 1.2 Default value changed from 1e-3 to 1e-4 for consistency with other linear models.",0.0001
,"solver solver: {'auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs'}, default='auto'Solver to use in the computational routines:- 'auto' chooses the solver automatically based on the type of data.- 'svd' uses a Singular Value Decomposition of X to compute the Ridge coefficients. It is the most stable solver, in particular more stable for singular matrices than 'cholesky' at the cost of being slower.- 'cholesky' uses the standard :func:`scipy.linalg.solve` function to obtain a closed-form solution.- 'sparse_cg' uses the conjugate gradient solver as found in :func:`scipy.sparse.linalg.cg`. As an iterative algorithm, this solver is more appropriate than 'cholesky' for large-scale data (possibility to set `tol` and `max_iter`).- 'lsqr' uses the dedicated regularized least-squares routine :func:`scipy.sparse.linalg.lsqr`. It is the fastest and uses an iterative procedure.- 'sag' uses a Stochastic Average Gradient descent, and 'saga' uses its improved, unbiased version named SAGA. Both methods also use an iterative procedure, and are often faster than other solvers when both n_samples and n_features are large. Note that 'sag' and 'saga' fast convergence is only guaranteed on features with approximately the same scale. You can preprocess the data with a scaler from :mod:`sklearn.preprocessing`.- 'lbfgs' uses L-BFGS-B algorithm implemented in :func:`scipy.optimize.minimize`. It can be used only when `positive` is True.All solvers except 'svd' support both dense and sparse data. However, only'lsqr', 'sag', 'sparse_cg', and 'lbfgs' support sparse input when`fit_intercept` is True... versionadded:: 0.17 Stochastic Average Gradient descent solver... versionadded:: 0.19 SAGA solver.",'auto'
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive.Only 'lbfgs' solver is supported in this case.",False
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag' or 'saga' to shuffle the data.See :term:`Glossary ` for details... versionadded:: 0.17 `random_state` to support Stochastic Average Gradient.",None


predicting from the Ridge model

In [269]:
y_pred_ridge = ridge_model.predict(x_test_scaled)

evaluating the model

In [270]:
from sklearn import metrics 
mse_ridge = metrics.mean_squared_error(y_test, y_pred_ridge)
rmse_ridge = np.sqrt(mse_ridge)
print("RMSE(ridge) :", rmse_ridge)

mae_ridge = metrics.mean_absolute_error(y_test, y_pred_ridge)
print("MAE(ridge) :", mae_ridge)

r2_score_ridge = metrics.r2_score(y_test, y_pred_ridge)
print("R2 score (ridge):", r2_score_ridge)

RMSE(ridge) : 957.6966261864015
MAE(ridge) : 491.20664656714905
R2 score (ridge): 0.5051564100858185


In [271]:
coeffecients_ridge = pd.DataFrame({
    'Features': x.columns,
    'coeffecients_ridge': ridge_model.coef_
})
print(coeffecients_ridge)

          Features  coeffecients_ridge
0        store_nbr           -2.884544
1      onpromotion          219.217637
2             year          -26.827102
3            month           16.246405
4              day          -21.272415
5          holiday            0.000000
6        day_lag_1         -147.078090
7       day_lag_30          -66.821664
8   rolling_mean_7         1315.726567
9    rolling_std_7         -754.999788
10       lag_1_dup         -147.078090
11      lag_30_dup          -66.821664
12     lag_1_noise            0.095754


**No matter how much i try to make the model different they are performing same may be because of the larger volume and clean dataset.**

# Lasso Regression

In [279]:
from sklearn.linear_model import Lasso
model_lasso = Lasso(alpha=1)

model_lasso.fit(x_train_scaled, y_train)

,"alpha alpha: float, default=1.0Constant that multiplies the L1 term, controlling regularizationstrength. `alpha` must be a non-negative float i.e. in `[0, inf)`.When `alpha = 0`, the objective is equivalent to ordinary leastsquares, solved by the :class:`LinearRegression` object. For numericalreasons, using `alpha = 0` with the `Lasso` object is not advised.Instead, you should use the :class:`LinearRegression` object.",1
,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"precompute precompute: bool or array-like of shape (n_features, n_features), default=FalseWhether to use a precomputed Gram matrix to speed upcalculations. The Gram matrix can also be passed as argument.For sparse input this option is always ``False`` to preserve sparsity.",False
,"copy_X copy_X: bool, default=TrueIf ``True``, X will be copied; else, it may be overwritten.",True
,"max_iter max_iter: int, default=1000The maximum number of iterations.",1000
,"tol tol: float, default=1e-4The tolerance for the optimization: if the updates are smaller or equal to``tol``, the optimization code checks the dual gap for optimality and continuesuntil it is smaller or equal to ``tol``, see Notes below.",0.0001
,"warm_start warm_start: bool, default=FalseWhen set to ``True``, reuse the solution of the previous call to fit asinitialization, otherwise, just erase the previous solution.See :term:`the Glossary `.",False
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive.",False
,"random_state random_state: int, RandomState instance, default=NoneThe seed of the pseudo random number generator that selects a randomfeature to update. Used when ``selection`` == 'random'.Pass an int for reproducible output across multiple function calls.See :term:`Glossary `.",None
,"selection selection: {'cyclic', 'random'}, default='cyclic'If set to 'random', a random coefficient is updated every iterationrather than looping over features sequentially by default. This(setting to 'random') often leads to significantly faster convergenceespecially when tol is higher than 1e-4.",'cyclic'


In [280]:
y_pred_lasso = model_lasso.predict(x_test_scaled)

In [281]:
from sklearn import metrics 
mse_lasso = metrics.mean_squared_error(y_test, y_pred_lasso)
rmse_lasso = np.sqrt(mse_lasso)
print("RMSE(lasso) :", rmse_lasso)

mae_lasso = metrics.mean_absolute_error(y_test, y_pred_lasso)
print("MAE(lasso) :", mae_lasso)

r2_score_lasso = metrics.r2_score(y_test, y_pred_lasso)
print("R2 score(lasso) :", r2_score_lasso)

RMSE(lasso) : 957.3232638689238
MAE(lasso) : 490.5132124071349
R2 score(lasso) : 0.5055421688543084


In [282]:
coeffecients_lasso = pd.DataFrame({
    'Features': x.columns,
    'coeffecients': model_lasso.coef_
})
print(coeffecients_lasso)

          Features  coeffecients
0        store_nbr     -1.798945
1      onpromotion    218.995853
2             year    -25.666560
3            month     -0.000000
4              day     -3.925426
5          holiday      0.000000
6        day_lag_1     -4.965402
7       day_lag_30    -39.704529
8   rolling_mean_7   1291.784109
9    rolling_std_7   -732.816219
10       lag_1_dup   -286.152052
11      lag_30_dup    -92.077378
12     lag_1_noise     -0.000000


**now because of the huge clean datasets even after adding corelation adn noise metrics didn't change**

* so far  the insights that we can get from the coeffecients of different linear regression models are:

month and holiday is not a good feture or doesn;t affect the accuracy of models so massively.